# MPR121 Data Recording Notebook

Author: Christopher Parker (parkecp@mail.uc.edu)

I've tried to make this as user-friendly as possible, but feel free to reach out with any questions.

# Imports, Board Configuration and Function Definitions

In [1]:
# Basic libraries
import os
os.environ['BLINKA_MPR121'] = '1'
os.environ['BLINKA_FT232H'] = '1'
import time
import datetime
from collections import deque

# For writing data to file
import h5py

# Libraries for FTDI and I2C
from pyftdi.i2c import I2cController
from pyftdi.usbtools import UsbTools

# Multiprocessing and Asynchronous Execution
from concurrent.futures import ThreadPoolExecutor
import asyncio

# Widgets
import ipywidgets as widgets
from IPython.display import display

# Graphing (for test buttons)
import matplotlib.pyplot as plt

## Device Configuration and Constant Declarations

Before we start recording, we need to configure the MPR121s.

Normally, this would be done automatically by the Adafruit Blinka package, but we cannot use that (because it only allows 1 FT232H at a time).
So instead, we have to use the pyftdi package (which is what Adafruit Blinka uses, also) and manually determine which registers need to be written/read.

Before we start recording, we need to configure the MPR121s.

Normally, this would be done automatically by the Adafruit Blinka package, but we cannot use that (because it only allows 1 FT232H at a time). So instead, we have to use the pyftdi package (which is what Adafruit Blinka uses, also) and manually determine which registers need to be written/read.

In [2]:
# Register Addresses
SOFT_RESET = 0x80
CONFIG = 0x5E
DATA = 0x04

# How many sensor samples we want to store before writing
HISTORY_SIZE = 100

# 12 channels per MPR121, just defining a constant
# for clarity
NUM_CHANNELS = 12

# Filename for saving data (placeholder, will be set below)
filename = ""

In [3]:
# Find all devices with the given vendor and product IDs
# These correspond to the FT232H
devices = UsbTools.find_all([(0x0403, 0x6014)])

NUM_BOARDS = len(devices)
NUM_SENSORS_TOTAL = NUM_BOARDS*12

In [4]:
devices

[(UsbDeviceDescriptor(vid=1027, pid=24596, bus=0, address=255, sn='FT232H1', index=None, description='\uffff\uffff\uffff\uffff\uffff\uffff'),
  1),
 (UsbDeviceDescriptor(vid=1027, pid=24596, bus=0, address=254, sn='FT232H0', index=None, description='\uffff\uffff\uffff\uffff\uffff\uffff'),
  1)]

In [14]:
# We will make lists of the I2C controller objects and port objects
# for each FT232H device
i2c_controllers = []
i2c_ports = []
for dev in devices:
    url = f"ftdi://ftdi:232h:{hex(dev[0].bus)}:{hex(dev[0].address)}/1"
    controller = I2cController()
    controller.configure(url)
    i2c_controllers.append(controller)
    port = controller.get_port(0x5A)
    # I really need to figure out a better way to determine the correct address,
    # because this looks silly
    try: # Try to read a byte to see if that's the right address
        port.read_from(0x04, 1)
    except: # If not, try again
        port = controller.get_port(0x5B)
    try:
        port.read_from(0x04, 1)
    except:
        port = controller.get_port(0x5C)
    try:
        port.read_from(0x04, 1)
    except:
        port = controller.get_port(0x5D)
    try:
        port.read_from(0x04,1)
    except:
        print('None of the MPR121 addresses are readable, is the FT232H in I2C mode?')
    i2c_ports.append(port)

In [15]:
# Loop over the FT232Hs, reset and configure each
for idx, port in enumerate(i2c_ports):
    # Write to Soft Reset Register (0x63 sends reset command)
    port.write_to(SOFT_RESET, b'\x63')
    # 0x5E is the configuration register, setting to 0x8F starts the MPR121
    # with the config used in the Adafruit library (if needed, I can figure
    # out alternative configurations)
    port.write_to(CONFIG, b'\x8F')
    
    # If we don't sleep here, it doesn't have time to start reading properly
    time.sleep(0.1)
    
    # Test that we have started the MPR121 (if it's not properly started,
    # reading from DATA will give an empty bytearray, so we check if that's
    # what we got back)
    cap = port.read_from(DATA, 24)
    if cap != bytearray(24):
        print(f'Board {idx} Configured')

Board 0 Configured
Board 1 Configured


## Recording Function and Loop

In [7]:
def record(i2c_port):
    """
    Reads 24 bytes (2 bytes per channel for 12 channels) from an MPR121 sensor 
    via the given pyftdi I2C port, and returns timestamp and capacitance data.
    """
    local_time_data = deque(maxlen=NUM_CHANNELS)
    local_cap_data = deque(maxlen=NUM_CHANNELS)

    # Read 24 bytes (2 bytes for each of the 12 channels).
    raw_buffer = i2c_port.read_from(DATA, 24)
    for chan in range(NUM_CHANNELS):
        # Combine two bytes (little-endian)
        value = raw_buffer[2 * chan] | (raw_buffer[2 * chan + 1] << 8)
        local_cap_data.append(value)
        local_time_data.append(time.time())
    return local_time_data, local_cap_data

In [8]:
debug_ctr = 0
async def record_sensors():
    """
    Asynchronous function for recording from all sensors and periodically
    writing to file. This is done asynchronously so that we are not blocking
    user inputs while recording (so the user can start/stop and add other data
    to the H5 file without stopping everything).
    """
    global debug_ctr
    board_time_data = {
        board: [deque(maxlen=HISTORY_SIZE) for _ in range(NUM_CHANNELS)]
        for board in range(NUM_BOARDS)
    }
    board_cap_data = {
        board: [deque(maxlen=HISTORY_SIZE) for _ in range(NUM_CHANNELS)]
        for board in range(NUM_BOARDS)
    }
    # Track how many reads we've done so that we can write at the appropriate
    # intervals
    loop_ctr = 0
    
    with ThreadPoolExecutor(max_workers=NUM_BOARDS) as executor:
        # Create a group for each board and initialize datasets.
        with h5py.File(filename, "w") as h5f:
            board_groups = {}
            for board in range(NUM_BOARDS):
                grp = h5f.create_group(f"board_{board}")
                for sensor in range(NUM_CHANNELS):
                    grp.create_group(f"sensor_{sensor}")
        while recording_all:
            # Because the while loop is computing stuff pretty much constantly,
            # it blocks the main execution thread for the notebook unless we
            # await for a moment here. This gives control back to the notebook,
            # and checks if the user has pressed any buttons since the last loop
            await asyncio.sleep(0)
            # Launch parallel sensor reads on all I2C ports.
            futures = [executor.submit(record, port) for port in i2c_ports]
            results = [future.result() for future in futures]  # Each result is (local_time_data, local_cap_data)
            # Append the data from each board to its corresponding deques.
            for board, (local_time, local_cap) in enumerate(results):
                for sensor in range(NUM_CHANNELS):
                    board_time_data[board][sensor].append(local_time[sensor])
                    board_cap_data[board][sensor].append(local_cap[sensor])
            if loop_ctr == HISTORY_SIZE:
                with h5py.File(filename, "r+") as h5f:
                    for board in range(NUM_BOARDS):
                        for sensor in range(NUM_CHANNELS):
                            group = h5f[f"board_{board}"][f"sensor_{sensor}"]
                            debug_ctr = board_time_data[board][sensor]
                            group.create_dataset("time_data", data=board_time_data[board][sensor], chunks=(HISTORY_SIZE,), maxshape=(None,))
                            group.create_dataset("cap_data", data=board_cap_data[board][sensor], chunks=(HISTORY_SIZE,), maxshape=(None,))
            elif loop_ctr != 0 and loop_ctr%HISTORY_SIZE == 0:
                tmp_ctr = loop_ctr - HISTORY_SIZE
                with h5py.File(filename, "r+") as h5f:
                    for board in range(NUM_BOARDS):
                        for sensor in range(NUM_CHANNELS): 
                            group = h5f[f"board_{board}"][f"sensor_{sensor}"]
                            group["time_data"].resize((tmp_ctr + HISTORY_SIZE,))
                            group["cap_data"].resize((tmp_ctr + HISTORY_SIZE,))
                            group["time_data"][tmp_ctr:tmp_ctr + HISTORY_SIZE] = board_time_data[board][sensor]
                            group["cap_data"][tmp_ctr:tmp_ctr + HISTORY_SIZE] = board_cap_data[board][sensor]
            loop_ctr += 1

## Widget Callback Functions

In [9]:
# --- Callback for the Main Start/Stop Button ---
def start_stop_all(button):
    global recording_all, recording_task, filename
    if not recording_all:
        recording_all = True
        # Define the filename here so that another recording session can
        # start without rerunning everything (although it would still be a
        # good idea, I'm doing this in case of user error)
        filename = f"raw_data_{datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.h5"
        button.description = "Stop Recording"
        recording_task = asyncio.create_task(record_sensors())
        with output_area:
            print("Started recording sensors asynchronously.")
    else:
        recording_all = False
        button.description = "Start Recording"
        with output_area:
            print("Stopping recording sensors.")
        if recording_task is not None:
            recording_task.cancel()
        # Make sure all sensor buttons are switched to the stopped state
        # (this will also trigger saving the stop_time for each sensor
        # that was not switched off manually)
        for row in sensor_rows:
            if isinstance(row[0], widgets.widget_bool.ToggleButton):
                row[0].value = False
                sensor_id = row[0].sensor_id
                board_id = sensor_id // 12
            if isinstance(row[2], widgets.widget_float.BoundedFloatText):
                start_vol = row[2].value
                stop_vol = row[3].value
            with h5py.File(filename, "r+") as h5f:
                # Try/except here in case the user never started/stopped some sensor and a group
                # was never created for it.
                try:
                    h5f[f"board_{board_id}"][f"sensor_{sensor_id}"].create_dataset("start_vol", data = start_vol)
                    h5f[f"board_{board_id}"][f"sensor_{sensor_id}"].create_dataset("stop_vol", data = stop_vol)
                except KeyError:
                    h5f[f"board_{board_id}"].create_group(f"sensor_{sensor_id}")
                    h5f[f"board_{board_id}"][f"sensor_{sensor_id}"].create_dataset("start_vol", data = start_vol)
                    h5f[f"board_{board_id}"][f"sensor_{sensor_id}"].create_dataset("stop_vol", data = stop_vol)

In [10]:
# --- Callback for Individual Sensor Start/Stop ---
def sensor_recording(sensor_id, starting):
    global start_except_ctr, stop_except_ctr
    with output_area:
        if starting:
            print(f"Sensor {sensor_id}: Recording start triggered.")
            # Set the board ID number based on the sensor ID,
            # sensors 0-11 are on board 0, etc.
            board_id = sensor_id // 12
            with h5py.File(filename, "r+") as h5f:
                try:
                    h5f[f"board_{board_id}"][f"sensor_{sensor_id%12}"].create_dataset("start_time", data=time.time())
                except ValueError as e:
                    start_except_ctr += 1
                    print("Attempted to re-record the start time after it has been set")
                    h5f[f"board_{board_id}"][f"sensor_{sensor_id%12}"].create_dataset(f"start_time{start_except_ctr}", data=time.time())
        else:
            print(f"Sensor {sensor_id}: Recording stop triggered.")
            board_id = sensor_id // 12
            with h5py.File(filename, "r+") as h5f:
                try:
                    h5f[f"board_{board_id}"][f"sensor_{sensor_id%12}"].create_dataset("stop_time", data=time.time())
                except ValueError as e:
                    stop_except_ctr += 1
                    print("Attempted to re-record the stop time after it has been set")
                    h5f[f"board_{board_id}"][f"sensor_{sensor_id%12}"].create_dataset(f"stop_time{stop_except_ctr}", data=time.time())

In [11]:
# --- Callback for Individual Sensor Test Buttons ---
def sensor_test(button):
    with graph_area:
        graph_area.clear_output()
        print(f"Sensor {button.sensor_id}: Test triggered - plotting raw data for confirmation.")
        board_id = (button.sensor_id) // 12
        try:
            with h5py.File(filename, "r") as h5f:
                last_20sec = h5f[f"board_{board_id}"][f"sensor_{button.sensor_id%12}"]["cap_data"][-1000:]
                fig, ax = plt.subplots()
                # Plotting the data; using the index as the x-axis values
                ax.plot(range(len(last_20sec)), last_20sec)
                ax.set_xlabel("Index")
                ax.set_ylabel("Sensor Data")
                ax.set_title(f"Sensor {button.sensor_id}: Last ~20 sec of raw data")
                plt.show()
        except KeyError:
            print("Please wait until at least 1000 reads have been completed before testing!")

In [12]:
# --- Callback for the Clear Output/Graph Buttons ---
def clear_output_callback(button):
    output_area.clear_output()
def clear_graph_callback(button):
    graph_area.clear_output()

# Start Here

In [13]:
# Define widgets

# Global flags and variables for async recording.
recording_all = False
recording_task = None

# Counters to check if we are setting multiple start/end times
# This is probably not necessary, but just in case someone makes a mistake and clicks
# the start or stop on the wrong sensor, we can allow saving multiple start and stop times
# instead of forcing the user to start over. It'll be a bit confusing later on, is the
# issue I see, if there are multiple start times and it's not carefully recorded which
# was correct.
start_except_ctr = 0
stop_except_ctr = 0

# --- Create the Output Widgets for Text and Graphing ---
output_area = widgets.Output(layout=widgets.Layout(overflow_y='scroll', height='500px', width='500px'))
graph_area = widgets.Output(layout=widgets.Layout(overflow_y='scroll', height='500px'))
outputs_hbox = widgets.HBox([
    output_area, graph_area,  
])

# --- Create the Main Start/Stop Button ---
all_devices_button = widgets.Button(
    description="Start Recording",
    button_style='primary',
    layout=widgets.Layout(width='400px', height='60px')
)
all_devices_button.on_click(start_stop_all)

# --- Create Sensor Rows: Each row includes a toggle button and a test button ---
sensor_rows = []
for i in range(NUM_SENSORS_TOTAL):
    # Create the sensor toggle button (starts as "Start").
    toggle_btn = widgets.ToggleButton(
        value=False,
        description=f"Sensor {i}: Start",
        layout=widgets.Layout(width='200px')
    )
    toggle_btn.sensor_id = i
    # Define the observer for the toggle button.
    def on_toggle(change, btn=toggle_btn):
        if change['new']:
            btn.description = f"Sensor {btn.sensor_id}: Stop"
            sensor_recording(btn.sensor_id, starting=True)
        else:
            btn.description = f"Sensor {btn.sensor_id}: Start"
            sensor_recording(btn.sensor_id, starting=False)
    toggle_btn.observe(on_toggle, names='value')

    # Create the sensor-specific test button.
    test_btn = widgets.Button(
        description="Test",
        button_style='info',
        layout=widgets.Layout(width='100px')
    )
    test_btn.sensor_id = i
    test_btn.on_click(sensor_test)

    start_vol_tinput = widgets.BoundedFloatText(
        value=0,
        min=0,
        max=100.0,
        step=0.01,
        layout=widgets.Layout(width='100px'),
    )
    start_vol_tinput.sensor_id = i
    stop_vol_tinput = widgets.BoundedFloatText(
        value=0,
        min=0,
        max=100.0,
        step=0.01,
        layout=widgets.Layout(width='100px'),
    )
    stop_vol_tinput.sensor_id = i

    # Combine the toggle and test buttons with the volume input boxes into a horizontal box.
    sensor_row = [toggle_btn, test_btn, start_vol_tinput, stop_vol_tinput]
    sensor_rows.append(sensor_row)

# --- Arrange the Sensor Rows into Columns ---
columns = []
for j in range(NUM_SENSORS_TOTAL // 12):
    sensors_subset = [item for row in sensor_rows[j * 12:(j + 1) * 12] for item in row]
    # Create GridBox layout
    grid_layout = widgets.Layout(
        display='grid',
        grid_template_columns='auto auto auto auto',
        #grid_gap='10px 10px',
        width='600px',
        justify_content='flex-start'
    )
    start_column_header = widgets.Label(
        value="Start Vol",
    )
    stop_column_header = widgets.Label(
        value="Stop Vol",
    )
    # I'm just using empty labels over the first 2 columns so that the start and stop volume labels are
    # aligned properly
    grid_w_headers = [widgets.Label(), widgets.Label(), start_column_header, stop_column_header, *sensors_subset]
    column = widgets.GridBox(grid_w_headers, layout=grid_layout)
    columns.append(column)
sensors_columns = widgets.HBox(columns)

# --- Create the Clear Output/Graph Buttons ---
clear_output_button = widgets.Button(
    description="Clear Output",
    button_style='warning'
)
# clear_output_button.layout.width = '150px'
# clear_output_button.layout.height = '40px'
clear_output_button.on_click(clear_output_callback)

clear_graph_button = widgets.Button(
    description="Clear Graph",
    button_style='warning'
)
# Set the layout properties after creation to avoid JSON serialization issues.
# clear_graph_button.layout.width = '150px'
# clear_graph_button.layout.height = '40px'
clear_graph_button.on_click(clear_graph_callback)

clear_buttons = widgets.HBox([
    clear_output_button, clear_graph_button,
])

# --- Combine Everything into the Final Layout ---
final_ui = widgets.VBox([
    all_devices_button,  # Top large start/stop-all button
    #column_headers,      # Text box to display "Start Vol" and "Stop Vol" above those columns
    sensors_columns,     # Sensor grid arranged in 3 columns
    clear_buttons,       # Buttons to clear the output/graph widgets
    outputs_hbox,        # Output/graph widgets side-by-side
])

# Display the complete UI.
display(final_ui)